<a href="https://colab.research.google.com/github/mgmills93/AEM_Spring_2019/blob/master/SecureCommunities_FinalProject/FinalProject_Hispanic_v0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Question 4:	Is the number of state SC arrests correlated with the state’s Hispanic population?
---
I will compare SC arrests by state from 2015-2017 with the percentage of that state's population that claimed some hispanic origin in 2016. 

This notebook will be updated with a pipeline of commands; I have been having trouble with the file so this one is a placeholder that arrives at the same dataset. (4/21/19)

In [8]:
import pandas as pd
import numpy as np
sc_arrests = pd.read_excel('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/enforcement/Secure_Communities_2017_Deportations.xlsx', header=4)
sum_arrests = sc_arrests.groupby('State').size().to_frame('Arrest_Number').reset_index()
print(sum_arrests.sample(5))


   State  Arrest_Number
45    TN           1820
33    NJ           1154
53    WV            259
0     AK             10
32    NH            114


In [9]:
total_pop = pd.read_csv('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/demographics/ACS_2016_Total_Pop_copy.csv', header=1)
print(total_pop.head(10))
total_pop.drop(total_pop.columns[[0, 1, 4]], axis=1, inplace=True)
# I will rename the columns for geography and total population
total_pop.rename(columns={'Geography':'State', 'Estimate; Total':'Total_Population'}, inplace=True)
print(total_pop.sample(5))
# Now I will convert the state names to the two-letter abbreviations to match my other dataset
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Colmbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',}

total_pop['State']=total_pop['State'].map(us_state_abbrev)
print(total_pop.sample(5))


            Id   Id2             Geography  Estimate; Total  \
0    0100000US   NaN         United States        318558162   
1  0400000US01   1.0               Alabama          4841164   
2  0400000US02   2.0                Alaska           736855   
3  0400000US04   4.0               Arizona          6728577   
4  0400000US05   5.0              Arkansas          2968472   
5  0400000US06   6.0            California         38654206   
6  0400000US08   8.0              Colorado          5359295   
7  0400000US09   9.0           Connecticut          3588570   
8  0400000US10  10.0              Delaware           934695   
9  0400000US11  11.0  District of Columbia           659009   

  Margin of Error; Total  
0                  *****  
1                  *****  
2                  *****  
3                  *****  
4                  *****  
5                  *****  
6                  *****  
7                  *****  
8                  *****  
9                  *****  
        S

In [10]:
hisp_total = pd.read_csv('https://raw.githubusercontent.com/mgmills93/AEM_Spring_2019/master/demographics/ACS_2016_Hisp_Pop_copy.csv', header=1)
hisp_total.drop(hisp_total.columns[[0, 1, 3, 4, 5, 6, 8]], axis=1, inplace=True)
# I will now add the missing states
missing_states = [pd.Series(['Alaska', 50162], index=hisp_total.columns) ,
                  pd.Series(['Montana', 36626], index=hisp_total.columns) ,
                  pd.Series(['North Dakota', 24744], index=hisp_total.columns), 
                  pd.Series(['South Dakota', 29901], index=hisp_total.columns), 
                  pd.Series(['Vermont', 11183], index=hisp_total.columns), 
                  pd.Series(['West Virginia', 27021], index=hisp_total.columns), 
                  pd.Series(['Wyoming', 56681], index=hisp_total.columns),
                  pd.Series(['Nebraska', 167405], index=hisp_total.columns)]
comp_hisp_total = hisp_total.append(missing_states, ignore_index=True)
# I want to rename the columns for geography and total population
comp_hisp_total.rename(columns={'Estimate; Total: - Hispanic or Latino:':'Hispanic_Population', 'Geography':'State'}, inplace=True)
print(comp_hisp_total.tail(5))
# I will now convert the state names to their respective two-letter codes
comp_hisp_total['State']=comp_hisp_total['State'].map(us_state_abbrev)
print(comp_hisp_total.sample(5))

            State  Hispanic_Population
48   South Dakota                29901
49        Vermont                11183
50  West Virginia                27021
51        Wyoming                56681
52       Nebraska               167405
   State  Hispanic_Population
22    MN               288460
23    MS                86733
38    TN               347442
13    IN               448079
34    OR               522568


In [13]:
pop_th3set = sum_arrests.merge(total_pop, on='State', how='inner')
final_hisp_set = pop_th3set.merge(comp_hisp_total, on='State', how='inner')
# I will now create a column %Hispanic by dividing Total 2016 Population by 2016 Hispanic Population
final_hisp_set['Percent_Hispanic'] = final_hisp_set['Hispanic_Population']/final_hisp_set['Total_Population']*100
final_hisp_set.sort_values('State')
print(final_hisp_set.head())
final_hisp_set.to_csv("hisp_set.csv")

  State  Arrest_Number  Total_Population  Hispanic_Population  \
0    AK             10            736855                50162   
1    AL           1173           4841164               199686   
2    AR            859           2968472               214544   
3    AZ          10568           6728577              2144775   
4    CA          24843          38654206             15280776   

   Percent_Hispanic  
0          6.807581  
1          4.124752  
2          7.227422  
3         31.875611  
4         39.531988  
